<a href="https://colab.research.google.com/github/jercash/build_a_star_overcashFINAL/blob/main/build_a_star_workshop_overcash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build-a-Star Workshop
*The most heat you'll ever make*

By Jeremy Overcash :)

In this notebook we will build a star using the Equations of Stellar Structure. This should work for a star of any mass on the main sequence.

Since this is a very difficult task, we will be making some preemptive assumptions. We will assume that the star...

- ...can transport all heat radiatively (convective heat transport is an entirely new beast)

- ...will use purely P-P Chain reactions to generate energy. The CNO cycle will be left out

- ...has a similar material composition and ionization as our Sun

- ...has a similar opacity to the Sun, which will remain constant throughout the star

The first step is to import libraries. We will need numpy, scpipy, and matplot:

In [35]:
import numpy as np
import scipy.constants as scp
import scipy.integrate as integrate
import matplotlib.pyplot as plt

# Variables and Constants

Next, we will define our variables and constants. $M$ can be altered by the viewer to change the mass of the star. Notice how there is only ONE thing the user can change; the goal behind this project is to show how *dependent* on its mass a star really is!

In [36]:
#user-defined variables:
M = 1 * 1.989e30 #kg, mass of the star. coverted from solar masses to kg

#constants:
G = 6.67e-11 #m^3/kg*s^2, gravitational constant
pi = scp.pi #our value of pi. this makes it easier to keep track of
X_fraction = 0.740 #in %, mass fraction of Hydrogen in our Sun
mu = 0.6 #unitless
Msolar = 1.989e30 #kg, mass of our Sun
Rsolar = 690000000 #m, radius of our Sun
opacity = 8 #m^2/kg, similar to the Sun's opacity
Eo = 1.08e-12 #W*m^3/kg^2, constant used in energy generation function
sigma = scp.sigma #W/m^2K^4, Stefan-Boltzmann constant
mh = scp.m_p #kg, mass of proton
k = scp.k #boltzmann constant

# Equations and Functions

We will be using many equations to build our star. Here is a list of their names:

- Hydrostatic Equilibrium

- Equation of State

- Mass Continuity

- Radiative Temperature Gradient

- Solar Luminosity

- Energy Generation

There's a lot here, so I will define them in comments as we go. 

First up, we will define an equation to find the radius of the star from the user-defined mass:

In [37]:
if (M / Msolar) == 1.0: #we want the one we actually know to be true!
  R = Rsolar
else:
  if (M / Msolar) > 1.66: #the rules for radius change as stars get large!
    R = 1.33*Rsolar*(M/Msolar)**0.555
  else:
    R = 1.06*Rsolar*(M/Msolar)**0.945

R = int(R)
#theta = np.linspace(0,R,360)

Now that we have our radius, we can write an equation to find density. Since density changes with radial distance, we must define:

In [38]:
#def density(r): 
density = 3*M / (4*pi*R**3)

That wasn't so bad! Now, let's get to the good stuff. We can find the mass internal to a radial distance using the Mass Continuity Equation. This is where we need to start integrating, since this equation traditionally gives us the *change* in mass:

In [39]:
def mass(r): #integral of mass continuity equation
  return 4*pi*r**3*density/3

print(mass(R))
##constants for integration:
#a = 0 #start. cannot use 0 because we will be dividing by this number
#N = 100 #number of slices

#def integral_mass(r): #integral of mass continuity equation
#  b = r
#  h = (b-a) / N 
#  x = np.linspace(a,b,N+1,endpoint=True)
#  y = mass(x)

#  plt.plot(x,y)
#  return h * (np.sum(y) - 0.5*(y[0]+y[-1]))

1.989e+30


Now that we have our mass continuity equation ready, we can set up our equation for Hydrostatic Equilibrium. This is *also* a "change-in" equation, so we need to find the integral as well.

In [40]:
def pressure(r): #integral of hydrostatic equilibrium
  return G*mass(r)/r

print(pressure(R))

192270000000.0


The next equations we need are our Luminosity equations. Before we can define the change in luminosity, we need to define the equation for core temperature of a star, which writes:

In [41]:
def Tcore(r):
  return mu*mh*pressure(r)/(density*k)

print(Tcore(R))

9668.929137488933


Next, the equation for energy generation. The equation we are using here is for P-P chain energy generation, rather than CNO cycle energy generation:

In [53]:
def Epp(r): 
  return Eo*density*X_fraction**2*(Tcore(r)**4)/1e11
  
print(Epp(R))

7.471359019250179e-05


Now we can define our Luminosity as:

In [59]:
def luminosity(r): #integral of the luminosity equation
  return 3*pi*r**3*density*Epp(r)

print(luminosity(R))

3.3436199450899356e+26


Finally, our radiative temperature gradient equation can be defined. This is written as:

In [60]:
def Trad(r):
  return opacity*density*luminosity(r)/(64*pi*sigma*r*Tcore(r)**3)

print(Trad(10))

2.3987424828340922e-17


# 